In [5]:
! pip3 install --upgrade google-cloud-aiplatform  \
                                 google-cloud-storage \
                                 kfp \
                                 google-cloud-pipeline-components

  Using cached google_cloud_storage-2.16.0-py2.py3-none-any.whl.metadata (6.1 kB)
  Using cached google_api_core-2.18.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl.metadata (541 bytes)
INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 100.0 MB/s eta 0:00:0000:01
Using cached google_cloud_storage-2.16.0-py2.py3-none-any.whl (125 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 27.5 MB/s eta 0:00:00
Using cached protobuf-4.25.3-cp37-abi3-manylinux2014_x86_64.whl (294 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: google-api-core
    Found e

In [134]:
import kfp
from kfp import compiler, dsl
from kfp.dsl import component, pipeline, Artifact, ClassificationMetrics, Input, Output, Model, Metrics
#from kfp.components import create_component_from_func
from google.cloud import aiplatform as aip
from typing import NamedTuple

from datetime import datetime

#import kfp.v2.dsl as dsl
#import google_cloud_pipeline_components as gcc_aip
from google_cloud_pipeline_components.v1.dataset import TabularDatasetCreateOp
from google_cloud_pipeline_components.v1.automl.training_job import AutoMLTabularTrainingJobRunOp
from google_cloud_pipeline_components.v1.endpoint import EndpointCreateOp, ModelDeployOp

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

In [112]:
PROJECT_ID = 'gcp-ccai-auto-ml-contactcenter'
REGION= "europe-west3"
REPO_NAME = "repo-demo3"
SERVICE_ACCOUNT = "944308723981-compute@developer.gserviceaccount.com"
BUCKET = "ccai-storage"
PIPELINE_NAME = "automl_pipeline"
YAML_NAME = f"{PIPELINE_NAME}.yml"
PIPELINE_ROOT = f"gs://{BUCKET}/pipeline_root/"
DISPLAY_NAME = PIPELINE_NAME.replace("_", "-")
NOTEBOOK = "automl"
DATANAME = "datasetnlp"
FOLDER = 'pipeline'
PROJECT_ID = 'gcp-ccai-auto-ml-contactcenter'
TABLE_ID = "stepfinalbq"
TEXT_COLUMN = 'body_pre'
LOCATION = "europe-west3"

#BQ_SOURCE = "bq://gcp-ccai-auto-ml-contactcenter.datasetnlp.stepfinalbq"
OUTPUT_PROCESSING = 'step1_pipeline.parquet'
OUTPUT_TOKENIZATION = 'step2_pipeline.parquet'
OUTPUT_SENTIMENT = 'step3_pipeline.parquet'
OUTPUT_MODERATE = 'step4_pipeline.parquet'
OUTPUT_ENTITIES = 'step5_pipeline.parquet'
OUTPUT_FINAL = 'step_final_bq.parquet'
ENDPOINT_NAME = 'automl_datasetnlp_endpoint1'

FILE_PATH = 'articlesoutput.parquet'
NUM_DOC = 10

In [113]:
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

URI = f"gs://{BUCKET}/{DATANAME}/models/{NOTEBOOK}"
DIR = f"temp/{NOTEBOOK}"

In [114]:
# Resources
DEPLOY_COMPUTE = 'n1-standard-4'

# Model Training
VAR_TARGET = 'topic'
VAR_OMIT = (
    'uri,url,date,body,time,dateTime,dateTimePub,lang,isDuplicate,dataType,sentiment,' +
    'eventUri,image,sharesFacebook,' +
    'sourceLocationLabel,categoryLabels,' +
    'categoryWeights,' +
    'alexaCountryRank,date_column,year,year_month,' +
    'num_documents,' +
    'PERSON,OTHER,ORGANIZATION,' +
    'EVENT,LOCATION,WORK_OF_ART,CONSUMER_GOOD,NUMBER,DATE,' +
    'NUMBER_mean_salience,' +
    'DATE_mean_salience,PRICE,ADDRESS,' +
    'ADDRESS_mean_salience,PHONE_NUMBER,PHONE_NUMBER_mean_salience'
)

COLUMN_TOPICK = "shares_scaled, body_pre"

In [115]:
VAR_OMIT

'uri,url,date,body,time,dateTime,dateTimePub,lang,isDuplicate,dataType,sentiment,eventUri,image,sharesFacebook,sourceLocationLabel,categoryLabels,categoryWeights,alexaCountryRank,date_column,year,year_month,num_documents,PERSON,OTHER,ORGANIZATION,EVENT,LOCATION,WORK_OF_ART,CONSUMER_GOOD,NUMBER,DATE,NUMBER_mean_salience,DATE_mean_salience,PRICE,ADDRESS,ADDRESS_mean_salience,PHONE_NUMBER,PHONE_NUMBER_mean_salience'

In [116]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'944308723981-compute@developer.gserviceaccount.com'

In [117]:
# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


In [118]:
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_NAME
!gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_NAME

In [119]:
aip.init(project=PROJECT_ID, staging_bucket=PIPELINE_ROOT, location=REGION)
bq = bigquery.Client()

In [120]:
!gcloud projects get-iam-policy $PROJECT_ID --filter="bindings.members:$SERVICE_ACCOUNT" --format='table(bindings.role)' --flatten="bindings[].members"

ROLE
organizations/329273198709/roles/Ruolopersonalizzato
organizations/329273198709/roles/Ruolopersonalizzato517
roles/aiplatform.user
roles/artifactregistry.admin
roles/artifactregistry.createOnPushWriter
roles/artifactregistry.writer
roles/bigquery.admin
roles/cloudbuild.builds.builder
roles/cloudtranslate.editor
roles/contactcenterinsights.editor
roles/containerregistry.ServiceAgent
roles/dataflow.admin
roles/dataflow.developer
roles/dataflow.worker
roles/datastore.user
roles/dialogflow.client
roles/pubsub.editor
roles/run.developer
roles/securesourcemanager.repoCreator
roles/securesourcemanager.repoWriter
roles/source.writer
roles/storage.objectAdmin
roles/viewer


In [121]:
!rm -rf {DIR}
!mkdir -p {DIR}
DIR

'temp/automl'

In [122]:
!rm -rf {URI}
!mkdir -p {URI}

In [123]:
URI

'gs://ccai-storage/datasetnlp/models/automl'

In [124]:
### Creazione componenti custom

In [125]:
@component(base_image=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/data_preparation:latest")

def data_preprocessing(
    bucket: str,
    file_path: str,
    folder: str,
    parquet_file_name: str, 
    processed_dataset: Output[Artifact]
):  
    import logging
    
    from processing.data_preparation import GCSParquetLoader
    
    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s\n')
    
    processor = GCSParquetLoader(bucket, file_path, folder, parquet_file_name)
    processed_dataset.uri = processor.process()
    print("--")


In [126]:
@component(base_image=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/data_tokenization:latest")

def data_tokenization(
    bucket: str,
    file_path: Input[Artifact],
    folder: str,
    parquet_file_name: str,
    tokenized_dataset: Output[Artifact]
):  
    import logging
    from processing.tokenization import TokenizationProcessor
    
    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s\n')
    
    processor = TokenizationProcessor(bucket, file_path.uri, folder, parquet_file_name)
    #processor.save_df_to_gcs_parquet()
    
    print("--")
    tokenized_dataset.uri = processor.process()


In [127]:
@component(base_image=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/data_sentiment:latest")

def data_sentiment(
    bucket: str,
    file_path: Input[Artifact],
    folder: str,
    parquet_file_name: str,
    text_column: str,
    num_doc: int,
    sentiment_dataset: Output[Artifact]
):  
    import logging
    from processing.sentiment import GCSSentimentAnalyzer

    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s\n')
    
    processor = GCSSentimentAnalyzer(bucket, file_path.uri, folder, parquet_file_name, 
                                    text_column, num_doc)
    
    
    print("--")
    sentiment_dataset.uri = processor.process()


In [128]:
@component(base_image=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/data_moderate:latest")

def data_moderate(
    bucket: str,
    file_path: Input[Artifact],
    folder: str,
    parquet_file_name: str,
    text_column: str,
    num_doc: int,
    moderate_dataset: Output[Artifact]
):  
    import logging
    from processing.moderate import GCSTextModerationLoader

    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s\n')
    
    processor = GCSTextModerationLoader(bucket, file_path.uri, folder, parquet_file_name, 
                                    text_column, num_doc)
    
    print("--")
    moderate_dataset.uri = processor.process()


In [129]:
@component(base_image=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/data_entities:latest")

def data_entities(
    bucket: str,
    file_path: Input[Artifact],
    folder: str,
    parquet_file_name: str,
    text_column: str,
    num_doc: int,
    entities_dataset: Output[Artifact]
):  
    import logging
    from processing.entities import GCSCEntityAnalyzer

    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s\n')
    print(f"------ file_path.uri = {file_path.uri}")
    processor = GCSCEntityAnalyzer(bucket, file_path.uri, folder, parquet_file_name, 
                                    text_column, num_doc)
    
    
    entities_dataset.uri = processor.process()
    print("--")


In [130]:
FILE_PATH="step5_pipeline.parquet"

@component(base_image=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPO_NAME}/data_bigquery:latest")

def data_bigquery(
    bucket: str,
    #file_path: Input[Artifact],
    file_path: str,
    folder: str,
    parquet_file_name: str,
    project_id: str,
    dataname: str,
    table_id: str,
    location: str,
    big_query_gcs: Output[Artifact]
)-> NamedTuple(
    "Outputs",
    [
        ("big_query_out", str)  # Return parameters
    ],
    
):  
    
    import logging
    from processing.bigquery import GCS_Bigquery

    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s\n')
    #print(f"------ file_path.uri = {file_path.uri}")
    print("--")
    processor = GCS_Bigquery(bucket, file_path, folder, parquet_file_name, 
                         project_id, dataname, table_id, location)
    
    #processor = GCS_Bigquery(bucket, file_path.uri, folder, parquet_file_name, 
    #                     project_id, dataname, table_id, location)
    
    
    bigquery_table_out = processor.upload_dataframe_to_bigquery()
    big_query_gcs.uri = f"bq://{project_id}.{dataname}.{table_id}"
    return (big_query_gcs.uri,)

In [146]:

OPTIMIZATION_PREDICTION_TYPE = "classification"
OPTIMIZATION_OBJECTIVE="minimize-log-loss"
BUDGET_MILLI_NODE_HOURS=1000
DISABLE_EARLY_STOPPING=False
PREDEFINED_SPLIT_COLUMN_NAME='split'
THRESHOLDS_DICT_STR = '0.95'

In [147]:
@component(
    base_image="gcr.io/deeplearning-platform-release/tf2-cpu.2-6:latest",
    packages_to_install=["google-cloud-aiplatform"],
)

def classification_model_eval_metrics(
    project: str,
    location: str,
    thresholds_dict_str: str,
    model: Input[Artifact],
    metrics: Output[Metrics],
    metricsc: Output[ClassificationMetrics],
) -> NamedTuple("Outputs", [("dep_decision", str)]):  # Return parameter.

    import json
    import logging

    from google.cloud import aiplatform as aip

    aip.init(project=PROJECT_ID, staging_bucket=PIPELINE_ROOT, location=REGION)
    

    def retrieve_existing_model_metrics() -> int:
        """
        Retrieve the auRoc_existing metric from the evaluation of the existing model.
        """
        # Retrieve the existing model and its evaluation
        models = aip.Model.list(filter=f'labels.notebook={NOTEBOOK}')
        model = models[1]
        evaluation = model.get_model_evaluation().to_dict()

        # Extract the log loss metric
        auRoc_existing = evaluation['metrics']['auRoc']

        return auRoc_existing


    # Fetch model eval info
    def get_eval_info(model):
        response = model.list_model_evaluations()
        metrics_list = []
        metrics_string_list = []
        for evaluation in response:
            evaluation = evaluation.to_dict()
            print("model_evaluation")
            print(" name:", evaluation["name"])
            print(" metrics_schema_uri:", evaluation["metricsSchemaUri"])
            metrics = evaluation["metrics"]
            for metric in metrics.keys():
                logging.info("metric: %s, value: %s", metric, metrics[metric])
            metrics_str = json.dumps(metrics)
            metrics_list.append(metrics)
            metrics_string_list.append(metrics_str)

        return (
            evaluation["name"],
            metrics_list,
            metrics_string_list,
        )

    # Use the given metrics threshold(s) to determine whether the model is
    # accurate enough to deploy.
    def classification_thresholds_check(metrics_dict, thresholds_dict):
        for k, v in thresholds_dict.items():
            logging.info("k {}, v {}".format(k, v))
            if k in ["auRoc", "auPrc"]:  # higher is better
                if metrics_dict[k] < v and metrics_dict[k] < retrieve_existing_model_metrics():   # if under threshold, and under previous model aurRoc don't deploy
                    logging.info("{} < {}; returning False".format(metrics_dict[k], v))
                    return False
        logging.info("threshold checks passed.")
        return True

    def log_metrics(metrics_list, metricsc):
        test_confusion_matrix = metrics_list[0]["confusionMatrix"]
        logging.info("rows: %s", test_confusion_matrix["rows"])

        # log the ROC curve
        fpr = []
        tpr = []
        thresholds = []
        for item in metrics_list[0]["confidenceMetrics"]:
            fpr.append(item.get("falsePositiveRate", 0.0))
            tpr.append(item.get("recall", 0.0))
            thresholds.append(item.get("confidenceThreshold", 0.0))
        print(f"fpr: {fpr}")
        print(f"tpr: {tpr}")
        print(f"thresholds: {thresholds}")
        metricsc.log_roc_curve(fpr, tpr, thresholds)

        # log the confusion matrix
        annotations = []
        for item in test_confusion_matrix["annotationSpecs"]:
            annotations.append(item["displayName"])
        logging.info("confusion matrix annotations: %s", annotations)
        metricsc.log_confusion_matrix(
            annotations,
            test_confusion_matrix["rows"],
        )

        # log textual metrics info as well
        for metric in metrics_list[0].keys():
            if metric != "confidenceMetrics":
                val_string = json.dumps(metrics_list[0][metric])
                metrics.log_metric(metric, val_string)

    logging.getLogger().setLevel(logging.INFO)

    # extract the model resource name from the input Model Artifact
    model_resource_path = model.metadata["resourceName"]
    logging.info("model path: %s", model_resource_path)

    # Get the trained model resource
    model = aip.Model(model_resource_path)

    # Get model evaluation metrics from the the trained model
    eval_name, metrics_list, metrics_str_list = get_eval_info(model)
    logging.info("got evaluation name: %s", eval_name)
    logging.info("got metrics list: %s", metrics_list)
    log_metrics(metrics_list, metricsc)

    thresholds_dict = json.loads(thresholds_dict_str)
    deploy = classification_thresholds_check(metrics_list[0], thresholds_dict)
    if deploy:
        dep_decision = "true"
    else:
        dep_decision = "false"
    logging.info("deployment decision is %s", dep_decision)

    return (dep_decision,)


compiler.Compiler().compile(
    classification_model_eval_metrics, "tabular_eval_component.yaml"
)

In [156]:

@kfp.dsl.pipeline(
    name=f'kfp-{NOTEBOOK}-{DATANAME}-{TIMESTAMP}',
    pipeline_root=URI+'/'+str(TIMESTAMP)+'/kfp/',
    description="Data preprocessing text and Training Automl topic classification"
)

def pipeline(
    project: str,
    dataname: str,
    display_name: str,
    endpoint_name: str,
    deploy_machine: str,
    var_target: str,
    var_omit: str,
    features: dict,
    labels: dict,
    bucket: str,  # Added type annotation
    file_path: str,  # Added type annotation
    folder: str,  # Added type annotation
    project_id: str,  # Added type annotation
    table_id: str,  # Added type annotation
    text_column: str,  # Added type annotation
    location: str,  # Added type annotation
    num_doc: int,  # Added type annotation
    output_processing: str,  # Added type annotation
    output_tokenization: str,  # Added type annotation
    output_sentiment: str,  # Added type annotation
    output_moderate: str,  # Added type annotation
    output_entities: str,  # Added type annotation
    output_final: str, # Added type annotation
    optimization_prediction_type: str,
    optimization_objective: str,
    budget_milli_node_hours: int,
    disable_early_stopping: bool,
    predefined_split_column_name: str,
    thresholds_dict_str : str

):

    
    bigquery_op = data_bigquery(
        bucket=bucket,
        #file_path=entities_op.outputs['entities_dataset'],
        file_path=file_path,
        folder=folder,
        parquet_file_name=output_final,
        project_id=project_id,
        dataname=dataname,
        table_id=table_id,
        location=location
    )
        
    # dataset
    dataset_create_op  = TabularDatasetCreateOp(
        project=project,
        display_name=display_name,
        bq_source=bigquery_op.outputs["big_query_out"],
        labels=labels,
        location=location
    )
    
    training_op = AutoMLTabularTrainingJobRunOp(
        project = project,
        display_name = display_name,
        optimization_prediction_type = optimization_prediction_type,
        optimization_objective=optimization_objective,
        budget_milli_node_hours = budget_milli_node_hours,
        disable_early_stopping=disable_early_stopping,
        column_specs = features,
        dataset = dataset_create_op.outputs['dataset'],
        target_column = var_target,
        predefined_split_column_name = 'split',
        labels = labels,
        location = location
    )
    
    model_eval_task = classification_model_eval_metrics(
        project=project,
        location=location,
        thresholds_dict_str=thresholds_dict_str,
        model=training_op.outputs["model"],
    )
    
    with dsl.If(
        model_eval_task.outputs["dep_decision"] == "true",
        name="deploy_decision",
    ):

        endpoint_op = EndpointCreateOp(
            project=project,
            location=location,
            labels = labels,
            display_name=endpoint_name,
        )
        # Endpoint: Deployment of Model
        deployment = ModelDeployOp(
            model = training_op.outputs["model"],
            endpoint = endpoint_op.outputs["endpoint"],
            dedicated_resources_min_replica_count = 1,
            dedicated_resources_max_replica_count = 1,
            traffic_split = {"0": 100},
            dedicated_resources_machine_type= deploy_machine
        )
        
        

In [157]:
compiler.Compiler().compile(
    pipeline_func = pipeline,
    package_path = f"{DIR}/{NOTEBOOK}.json"
)

In [158]:
!gsutil cp {DIR}/{NOTEBOOK}.json {URI}/{TIMESTAMP}/kfp/

Copying file://temp/automl/automl.json [Content-Type=application/json]...
/ [1 files][ 64.9 KiB/ 64.9 KiB]                                                
Operation completed over 1 objects/64.9 KiB.                                     


In [159]:
# get feature names
query = f"SELECT * FROM {DATANAME}.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '{TABLE_ID}'"
schema = bq.query(query).to_dataframe()
OMIT = VAR_OMIT.split(",") + [VAR_TARGET, 'split']
features = schema[~schema.column_name.isin(OMIT)].column_name.tolist()
features = dict.fromkeys(features, 'auto')

In [160]:
query2 = f"SELECT * FROM {DATANAME}.{TABLE_ID}"
df2 = bq.query(query2).to_dataframe()
df2.columns

Index(['uri', 'url', 'title', 'body', 'date', 'time', 'dateTime',
       'dateTimePub', 'lang', 'isDuplicate', 'dataType', 'sentiment',
       'eventUri', 'relevance', 'image', 'authors', 'sharesFacebook',
       'sourceTitle', 'sourceLocationLabel', 'categoryLabels',
       'categoryWeights', 'importanceRank', 'alexaGlobalRank',
       'alexaCountryRank', 'date_column', 'year', 'month', 'year_month',
       'topic', 'split', 'shares_scaled', 'body_pre', 'score', 'magnitude',
       'num_documents', 'Toxic', 'Insult', 'Profanity', 'Derogatory', 'Sexual',
       'Death_Harm__Tragedy', 'Violent', 'Firearms__Weapons', 'Public_Safety',
       'Health', 'Religion__Belief', 'Illicit_Drugs', 'War__Conflict',
       'Politics', 'Finance', 'Legal', 'PERSON', 'OTHER', 'ORGANIZATION',
       'EVENT', 'LOCATION', 'WORK_OF_ART', 'CONSUMER_GOOD', 'NUMBER',
       'PERSON_mean_salience', 'OTHER_mean_salience',
       'ORGANIZATION_mean_salience', 'EVENT_mean_salience',
       'LOCATION_mean_salience'

In [161]:
features

{'title': 'auto',
 'relevance': 'auto',
 'authors': 'auto',
 'sourceTitle': 'auto',
 'importanceRank': 'auto',
 'alexaGlobalRank': 'auto',
 'month': 'auto',
 'shares_scaled': 'auto',
 'body_pre': 'auto',
 'score': 'auto',
 'magnitude': 'auto',
 'Toxic': 'auto',
 'Insult': 'auto',
 'Profanity': 'auto',
 'Derogatory': 'auto',
 'Sexual': 'auto',
 'Death_Harm__Tragedy': 'auto',
 'Violent': 'auto',
 'Firearms__Weapons': 'auto',
 'Public_Safety': 'auto',
 'Health': 'auto',
 'Religion__Belief': 'auto',
 'Illicit_Drugs': 'auto',
 'War__Conflict': 'auto',
 'Politics': 'auto',
 'Finance': 'auto',
 'Legal': 'auto',
 'PERSON_mean_salience': 'auto',
 'OTHER_mean_salience': 'auto',
 'ORGANIZATION_mean_salience': 'auto',
 'EVENT_mean_salience': 'auto',
 'LOCATION_mean_salience': 'auto',
 'WORK_OF_ART_mean_salience': 'auto',
 'CONSUMER_GOOD_mean_salience': 'auto',
 'PRICE_mean_salience': 'auto'}

In [162]:
pipeline = aip.PipelineJob(
    display_name = f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
    template_path = f"{URI}/{TIMESTAMP}/kfp/{NOTEBOOK}.json",
    pipeline_root=PIPELINE_ROOT,
    parameter_values = {
        "project" : PROJECT_ID,
        "dataname" : DATANAME,
        "display_name" : f'{NOTEBOOK}_{DATANAME}_{TIMESTAMP}',
        "endpoint_name" : ENDPOINT_NAME,
        "deploy_machine" : DEPLOY_COMPUTE,
        "var_target" : VAR_TARGET,
        "var_omit" : VAR_OMIT,
        "thresholds_dict_str": THRESHOLDS_DICT_STR,
        "optimization_prediction_type": OPTIMIZATION_PREDICTION_TYPE,
        "optimization_objective": OPTIMIZATION_OBJECTIVE,
        "budget_milli_node_hours": BUDGET_MILLI_NODE_HOURS,
        "disable_early_stopping": DISABLE_EARLY_STOPPING,
        "predefined_split_column_name": PREDEFINED_SPLIT_COLUMN_NAME,
        "features" : {'title': 'text',
                     'relevance': 'numeric',
                     'authors': 'categorical',
                     'sourceTitle': 'categorical',
                     'importanceRank': 'numeric',
                     'alexaGlobalRank': 'numeric',
                     'month': 'categorical',
                     'shares_scaled': 'numeric',
                     'body_pre': 'text',
                     'score': 'numeric',
                     'magnitude': 'numeric',
                     'Toxic': 'numeric',
                     'Insult': 'numeric',
                     'Profanity': 'numeric',
                     'Derogatory': 'numeric',
                     'Sexual': 'numeric',
                     'Death_Harm__Tragedy': 'numeric',
                     'Violent': 'numeric',
                     'Firearms__Weapons': 'numeric',
                     'Public_Safety': 'numeric',
                     'Health': 'numeric',
                     'Religion__Belief': 'numeric',
                     'Illicit_Drugs': 'numeric',
                     'War__Conflict': 'numeric',
                     'Politics': 'numeric',
                     'Finance': 'numeric',
                     'Legal': 'numeric',
                     'PERSON_mean_salience': 'numeric',
                     'OTHER_mean_salience': 'numeric',
                     'ORGANIZATION_mean_salience': 'numeric',
                     'EVENT_mean_salience': 'numeric',
                     'LOCATION_mean_salience': 'numeric',
                     'WORK_OF_ART_mean_salience': 'numeric',
                     'CONSUMER_GOOD_mean_salience': 'numeric',
                     'PRICE_mean_salience': 'numeric'},
        "labels" : {'notebook': NOTEBOOK},
        "bucket" : BUCKET,
        "file_path": FILE_PATH,
        "folder": FOLDER,
        "project_id": PROJECT_ID,
        "table_id": TABLE_ID,
        "text_column": TEXT_COLUMN,
        "location": LOCATION,
        "num_doc": NUM_DOC,
        "output_processing": OUTPUT_PROCESSING,
        "output_tokenization": OUTPUT_TOKENIZATION,
        "output_sentiment": OUTPUT_SENTIMENT,
        "output_moderate": OUTPUT_MODERATE,
        "output_entities": OUTPUT_ENTITIES,
        "output_final": OUTPUT_FINAL

    },
    labels = {'notebook': NOTEBOOK},
    enable_caching=True
)



In [163]:
pipeline.submit(service_account=SERVICE_ACCOUNT)

Creating PipelineJob
PipelineJob created. Resource name: projects/944308723981/locations/europe-west3/pipelineJobs/kfp-automl-datasetnlp-20240403102909-20240403112252
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/944308723981/locations/europe-west3/pipelineJobs/kfp-automl-datasetnlp-20240403102909-20240403112252')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/europe-west3/pipelines/runs/kfp-automl-datasetnlp-20240403102909-20240403112252?project=944308723981


Visual Representation of the pipeline can be viewed in the colsole:

In [79]:
print(f"Review the Pipeline as it runs here:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/pipelines/runs/{pipeline.resource_name.split('/')[-1]}?project={PROJECT_ID}")

Review the Pipeline as it runs here:
https://console.cloud.google.com/vertex-ai/locations/europe-west3/pipelines/runs/kfp-automl-datasetnlp-20240325224945-20240325225059?project=gcp-ccai-auto-ml-contactcenter


In [164]:
endpoint_name="automl_datasetnlp_20240324221421"

Retrieve the pipeline information:

In [171]:
endpoints_aip = aip.Endpoint.list(filter=("display_name={}").format(endpoint_name),
                                 location=LOCATION)
endpoints_aip

[]

In [ ]:
endpoint = None if len(endpoints_aip)==0 else endpoints_aip[0]

if not endpoint:
    endpoint = aiplatform.Endpoint.create(
         display_name=endpoint_name,
         project=project_id,
         location=location
     )

In [15]:
aip.get_pipeline_df(pipeline = f'kfp-{NOTEBOOK}-{DATANAME}-{TIMESTAMP}')

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/google/api_core/grpc_helpers.py", line 65, in error_remapped_callable
    if not hasattr(callable_, "__name__"):
  File "/opt/conda/lib/python3.10/site-packages/grpc/_channel.py", line 1160, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "/opt/conda/lib/python3.10/site-packages/grpc/_channel.py", line 1003, in _end_unary_response_blocking
    raise _InactiveRpcError(state)  # pytype: disable=not-instantiable
grpc._channel._InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Resource not found.; GetContext is unable to find context resource with name: projects/944308723981/locations/us-central1/metadataStores/default/contexts/kfp-automl-datasetnlp-20240403074448"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:142.250.186.138:443 {created_time:"2024-04-03T07:45:24.768020842+00:00", grpc_status:5, gr


### EVALUATION
Although the aforementioned model was trained utilizing AutoML via the API, users can still assess the evaluation metrics directly within the Google Cloud Console. By navigating to the Models section of the Vertex AI service and selecting the model, users will be presented with comprehensive evaluation metrics accompanied by various helpful visual aids.

Additionally, users have the option to retrieve the evaluation metrics for their model using the API. The following section outlines the process of utilizing the API for this purpose.


Get the Model:

In [12]:
NOTEBOOK

'automl'

In [66]:
models = aip.Model.list(filter=f'labels.notebook={NOTEBOOK}')
models

 resource name: projects/944308723981/locations/europe-west3/models/2095625182075944960,
 resource name: projects/944308723981/locations/europe-west3/models/8349999164586721280,
 resource name: projects/944308723981/locations/europe-west3/models/7334437448614674432]

In [67]:
model = models[0]
model.resource_name

'projects/944308723981/locations/europe-west3/models/2095625182075944960'

In [76]:
def get_eval_info(model):
    response = model.list_model_evaluations()
    metrics_list = []
    metrics_string_list = []
    for evaluation in response:
        evaluation = evaluation.to_dict()
        print("model_evaluation")
        print(" name:", evaluation["name"])
        print(" metrics_schema_uri:", evaluation["metricsSchemaUri"])
        metrics = evaluation["metrics"]
        #for metric in metrics.keys():
            #print("metric: %s, value: %s", metric, metrics[metric])
        metrics_str = json.dumps(metrics)
        metrics_list.append(metrics)
        metrics_string_list.append(metrics_str)

    return (
        evaluation["name"],
        metrics_list,
        metrics_string_list,
    )



In [77]:
eval_name, metrics_list, metrics_str_list = get_eval_info(model=model)
print("got evaluation name: %s", eval_name)

model_evaluation
 name: projects/944308723981/locations/europe-west3/models/2095625182075944960@1/evaluations/4587355914663221019
 metrics_schema_uri: gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml
got evaluation name: %s projects/944308723981/locations/europe-west3/models/2095625182075944960@1/evaluations/4587355914663221019


In [32]:
model.outputs["log_loss"]

AttributeError: 'Model' object has no attribute 'outputs'

Retrives the aggregate model evalution metrics for the model as a whole.  

Either:
- First, use `model.list_model_evaluations()` to retrieve the evaluation id, then use `model.get_model_evaluation(evaluation_id = )` for the evaluation id
- Or, use `.get_model_evaluation()` and it will retrieve the first model evaluation

In [79]:
evaluation = model.get_model_evaluation().to_dict() # get first evaluation

In [80]:
evaluation.keys()

dict_keys(['name', 'metricsSchemaUri', 'metrics', 'createTime', 'sliceDimensions', 'modelExplanation'])

In [81]:
evaluation['metrics'].keys()

dict_keys(['auPrc', 'auRoc', 'logLoss', 'confusionMatrix', 'confidenceMetrics'])

In [87]:
evaluation['metrics']['auRoc']

0.95601124

In [85]:
evaluation['metrics']['confidenceMetrics'][2]

{'falseNegativeCount': '1',
 'recallAt1': 0.7526096,
 'falsePositiveRateAt1': 0.035341486,
 'truePositiveCount': '957',
 'recall': 0.99895614,
 'trueNegativeCount': '1723',
 'confidenceThreshold': 0.005,
 'falsePositiveRate': 0.7430659,
 'precisionAt1': 0.7526096,
 'confusionMatrix': {'rows': [[13.0, 1.0, 1.0, 3.0, 1.0, 0.0, 5.0, 7.0, 0.0],
   [0.0, 76.0, 0.0, 0.0, 4.0, 0.0, 3.0, 8.0, 0.0],
   [0.0, 0.0, 2.0, 0.0, 1.0, 1.0, 0.0, 4.0, 0.0],
   [2.0, 1.0, 0.0, 247.0, 0.0, 4.0, 19.0, 16.0, 0.0],
   [0.0, 2.0, 1.0, 1.0, 40.0, 0.0, 4.0, 8.0, 0.0],
   [0.0, 0.0, 1.0, 0.0, 0.0, 9.0, 0.0, 0.0, 0.0],
   [1.0, 3.0, 0.0, 49.0, 0.0, 5.0, 101.0, 26.0, 0.0],
   [0.0, 7.0, 0.0, 24.0, 8.0, 2.0, 14.0, 233.0, 0.0],
   [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],
  'annotationSpecs': [{'displayName': 'Technology', 'id': '0'},
   {'displayName': 'Sports', 'id': '1'},
   {'displayName': 'Science', 'id': '2'},
   {'displayName': 'Politics', 'id': '3'},
   {'displayName': 'Health', 'id': '4'},
   {'displa

In [88]:
print(f"Review this model in the console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}/versions/{model.version_id}/evaluations/{evaluation['name'].split('/')[-1]}?project={PROJECT_ID}")

Review this model in the console:
https://console.cloud.google.com/vertex-ai/locations/europe-west3/models/2095625182075944960/versions/1/evaluations/4587355914663221019?project=gcp-ccai-auto-ml-contactcenter


In [89]:
for i in range(len(evaluation['metrics']['confusionMatrix']['annotationSpecs'])):
    print('True Label = ', evaluation['metrics']['confusionMatrix']['annotationSpecs'][i]['displayName'], ' has Predicted labels = ', evaluation['metrics']['confusionMatrix']['rows'][i])

True Label =  Technology  has Predicted labels =  [13.0, 1.0, 1.0, 3.0, 1.0, 0.0, 5.0, 7.0, 0.0]
True Label =  Sports  has Predicted labels =  [0.0, 76.0, 0.0, 0.0, 4.0, 0.0, 3.0, 8.0, 0.0]
True Label =  Science  has Predicted labels =  [0.0, 0.0, 2.0, 0.0, 1.0, 1.0, 0.0, 4.0, 0.0]
True Label =  Politics  has Predicted labels =  [2.0, 1.0, 0.0, 247.0, 0.0, 4.0, 19.0, 16.0, 0.0]
True Label =  Health  has Predicted labels =  [0.0, 2.0, 1.0, 1.0, 40.0, 0.0, 4.0, 8.0, 0.0]
True Label =  Environment  has Predicted labels =  [0.0, 0.0, 1.0, 0.0, 0.0, 9.0, 0.0, 0.0, 0.0]
True Label =  Business  has Predicted labels =  [1.0, 3.0, 0.0, 49.0, 0.0, 5.0, 101.0, 26.0, 0.0]
True Label =  Arts and Entertainment  has Predicted labels =  [0.0, 7.0, 0.0, 24.0, 8.0, 2.0, 14.0, 233.0, 0.0]
True Label =  DROPPED  has Predicted labels =  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [91]:
model_client = aip.gapic.ModelServiceClient(
    client_options = {
        'api_endpoint' : f'{REGION}-aiplatform.googleapis.com'
    }
)

In [92]:
slices = model_client.list_model_evaluation_slices(parent = evaluation['name'])

In [93]:
for slice in slices:
    print('Label = ', slice.slice_.value, 'has logLoss = ', slice.metrics['logLoss'])

Label =  Science has logLoss =  0.04694494
Label =  Arts and Entertainment has logLoss =  0.33384135
Label =  Business has logLoss =  0.3318977
Label =  Politics has logLoss =  0.26495764
Label =  Sports has logLoss =  0.101992965
Label =  Health has logLoss =  0.08870014
Label =  Technology has logLoss =  0.093657166
Label =  Environment has logLoss =  0.037851635


---
## Prediction

### Prepare a record for prediction: instance and parameters lists

In [99]:
VAR_OMIT

'uri,url,date,body,time,dateTime,dateTimePub,lang,isDuplicate,dataType,sentiment,eventUri,image,sharesFacebook,sourceLocationLabel,categoryLabels,categoryWeights,alexaCountryRank,date_column,year,year_month,num_documents,PERSON,OTHER,ORGANIZATION,EVENT,LOCATION,WORK_OF_ART,CONSUMER_GOOD,NUMBER,DATE,NUMBER_mean_salience,DATE_mean_salience,PRICE,ADDRESS,ADDRESS_mean_salience,PHONE_NUMBER,PHONE_NUMBER_mean_salience'

In [103]:
# Remove the duplicate date from the string
VAR_OMIT_rev = VAR_OMIT.replace("DATE,", "")

print(VAR_OMIT_rev)

uri,url,date,body,time,dateTime,dateTimePub,lang,isDuplicate,dataType,sentiment,eventUri,image,sharesFacebook,sourceLocationLabel,categoryLabels,categoryWeights,alexaCountryRank,date_column,year,year_month,num_documents,PERSON,OTHER,ORGANIZATION,EVENT,LOCATION,WORK_OF_ART,CONSUMER_GOOD,NUMBER,NUMBER_mean_salience,DATE_mean_salience,PRICE,ADDRESS,ADDRESS_mean_salience,PHONE_NUMBER,PHONE_NUMBER_mean_salience


In [107]:
pred = bq.query(
    query = f"""
        SELECT * EXCEPT({VAR_TARGET}, split, {VAR_OMIT_rev})
        FROM {DATANAME}.{TABLE_ID}
        WHERE split='TEST'
        LIMIT 10
    """
).to_dataframe()

In [151]:
pred.columns

Index(['title', 'relevance', 'authors', 'sourceTitle', 'importanceRank',
       'alexaGlobalRank', 'month', 'shares_scaled', 'body_pre', 'score',
       'magnitude', 'Toxic', 'Insult', 'Profanity', 'Derogatory', 'Sexual',
       'Death_Harm__Tragedy', 'Violent', 'Firearms__Weapons', 'Public_Safety',
       'Health', 'Religion__Belief', 'Illicit_Drugs', 'War__Conflict',
       'Politics', 'Finance', 'Legal', 'PERSON_mean_salience',
       'OTHER_mean_salience', 'ORGANIZATION_mean_salience',
       'EVENT_mean_salience', 'LOCATION_mean_salience',
       'WORK_OF_ART_mean_salience', 'CONSUMER_GOOD_mean_salience',
       'PRICE_mean_salience'],
      dtype='object')

Change the columns type for the autoML

In [140]:
pred['relevance'] = pred['relevance'].astype(str)
pred['importanceRank'] = pred['importanceRank'].astype(str)
pred['alexaGlobalRank'] = pred['alexaGlobalRank'].astype(str)
pred['month'] = pred['month'].astype(str)

newobs = pred.to_dict(orient='records')
newobs[0]

{'title': 'GF, Varrese sotto attacco: pesanti critiche da due ex Vipponi',
 'relevance': '1',
 'authors': 'Debora Manzoli',
 'sourceTitle': 'libero.it',
 'importanceRank': '1000000',
 'alexaGlobalRank': '0',
 'month': '12',
 'shares_scaled': -0.28,
 'body_pre': "Scrittrice , copywriter , editor pubblicista mantovana , laureata Lettere , Cinema Tv . due libri all'attivo ama scrittura follia . L'ultima puntata Grande Fratello visto protagonisti Massimiliano Varrese , ripreso Alfonso Signorini brutti atteggiamenti confronti Beatrice Luzzi . molti web infatti richiesto squalifica Varrese , stanchi atteggiamenti troppo aggressivi , gieffino fine ' cavata solamente scuse Beatrice pubblico ramanzina presentatore . , però , andato giù proprio , due ex vipponi Grande Fratello , passato stati squalificati gioco , passati all'attacco . Ecco tratta cosa detto . Dopo ramanzina Alfonso Signorini diretta , Varrese ammesso propri errori fine ' cavata ( sempre resto ) semplice ammonizione . `` uomo dev

In [141]:
instances = [json_format.ParseDict(newob, Value()) for newob in newobs]

### Get Predictions: Python Client

In [142]:
aip.Endpoint.list(filter=f'labels.notebook={NOTEBOOK}')

 resource name: projects/944308723981/locations/europe-west3/endpoints/1221042847017336832,
 resource name: projects/944308723981/locations/europe-west3/endpoints/1242434945247346688,
 resource name: projects/944308723981/locations/europe-west3/endpoints/6842661081882558464,
 resource name: projects/944308723981/locations/europe-west3/endpoints/238132228343726080]

In [143]:
endpoint = aip.Endpoint.list(filter=f'labels.notebook={NOTEBOOK}')[0]
endpoint.display_name

'automl_datasetnlp_20240325224945'

In [144]:
prediction = endpoint.predict(instances = instances) # or instances = newobs
prediction.predictions[0]

{'classes': ['Technology',
  'Sports',
  'Science',
  'Politics',
  'Health',
  'Environment',
  'Business',
  'Arts and Entertainment'],
 'scores': [0.01056508254259825,
  0.01481661759316921,
  0.00365602714009583,
  0.01545155979692936,
  0.001204681699164212,
  0.001032748143188655,
  0.01098049152642488,
  0.9422927498817444]}

### Get Predictions: REST

In [146]:
with open(f'{DIR}/request.json','w') as file:
    file.write(json.dumps({"instances": [newobs[0]]}))

In [147]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @{DIR}/request.json \
https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}:predict

{
  "predictions": [
    {
      "scores": [
        0.01056508533656597,
        0.014816612936556339,
        0.0036560278385877609,
        0.015451557002961641,
        0.001204681931994855,
        0.0010327481431886549,
        0.01098048780113459,
        0.94229274988174438
      ],
      "classes": [
        "Technology",
        "Sports",
        "Science",
        "Politics",
        "Health",
        "Environment",
        "Business",
        "Arts and Entertainment"
      ]
    }
  ],
  "deployedModelId": "1758308208813801472",
  "model": "projects/944308723981/locations/europe-west3/models/2095625182075944960",
  "modelDisplayName": "automl_datasetnlp_20240325224945",
  "modelVersionId": "1"
}
